In [1]:
import os
%pwd

'/home/kartik-agarwal/Documents/E2E_IMDB_50K_MovieReview/E2E_IMDB_50K_MovieReview/research'

In [2]:
os.chdir("../")
%pwd

'/home/kartik-agarwal/Documents/E2E_IMDB_50K_MovieReview/E2E_IMDB_50K_MovieReview'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    dataset_name: str


In [4]:
from E2E_IMDB_50K_MovieReview.src.CNNClassifier import *
from E2E_IMDB_50K_MovieReview.src.CNNClassifier.utils.common import read_yaml, create_directories

In [5]:
from E2E_IMDB_50K_MovieReview.src.CNNClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH


class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            dataset_name= config.dataset_name
        )

        return data_ingestion_config


In [6]:
import glob
import os
import urllib.request as request
import zipfile
from E2E_IMDB_50K_MovieReview.src.CNNClassifier import logger
from E2E_IMDB_50K_MovieReview.src.CNNClassifier.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        dataset_name = self.config.dataset_name
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        csv_files = glob.glob(os.path.join(unzip_path, "*.csv"))
        print(f"Found {len(csv_files)} CSV files.")
        for old_path in csv_files:
            # Get the file name from the full path
            file_name = os.path.basename(old_path)

            # Define your new name convention
            new_path = os.path.join(unzip_path, dataset_name)
            # Perform the rename
            try:
                os.rename(old_path, new_path)
                print(f"Renamed: {file_name} -> {dataset_name}")
            except OSError as e:
                print(f"Error renaming {file_name}: {e}")


In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-02-06 13:23:55,058: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-02-06 13:23:55,061: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-06 13:23:55,062: INFO: common: created directory at: artifacts]
[2026-02-06 13:23:55,063: INFO: common: created directory at: artifacts/data_ingestion]
[2026-02-06 13:23:55,063: INFO: 2145588375: File already exists of size: ~ 26331 KB]
Found 2 CSV files.
Renamed: data.csv -> data.csv
Renamed: IMDB Dataset.csv -> data.csv
